<a href="https://colab.research.google.com/github/anujkadu/InfinityPool-Work/blob/main/Infinity_Pool_TensorFlow_Auto.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Generating Dataset

In [1]:
import numpy as np
import pandas as pd
import tensorflow as tf
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split

# Generate synthetic FinTech data
np.random.seed(42)
N = 1000

df = pd.DataFrame({
    'age': np.random.randint(21, 60, N),
    'income': np.random.randint(20000, 150000, N),
    'credit_score': np.random.randint(300, 900, N),
    'loan_amount': np.random.randint(50000, 1000000, N),
    'loan_term': np.random.choice([12, 24, 36, 48, 60], N),
    'past_defaults': np.random.poisson(0.5, N)
})

# Define label rule
df['will_default'] = (
    (df['credit_score'] < 600).astype(int) |
    ((df['loan_amount'] / df['income']) > 10).astype(int) |
    (df['past_defaults'] > 2).astype(int)
)

# Separate features and target
X = df.drop("will_default", axis=1).values
y = df["will_default"].values

# Normalize features
scaler = MinMaxScaler()
X_scaled = scaler.fit_transform(X)

# Train/test split
X_train, X_test, y_train, y_test = train_test_split(
    X_scaled, y, test_size=0.2, random_state=42
)

# Convert to tensors
X_train = tf.convert_to_tensor(X_train, dtype=tf.float32)
X_test = tf.convert_to_tensor(X_test, dtype=tf.float32)
y_train = tf.convert_to_tensor(y_train, dtype=tf.float32)
y_test = tf.convert_to_tensor(y_test, dtype=tf.float32)


Implementing neural layers and activation function

In [2]:
from tensorflow.keras import Sequential
from tensorflow.keras.layers import Dense

# Build model (6 input features → 8 neurons → 4 → 1)
model = Sequential([
    Dense(8, activation='relu', input_shape=(6,)),
    Dense(4, activation='relu'),
    Dense(1, activation='sigmoid')  # Output: probability of default
])


/usr/local/lib/python3.11/dist-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


# Initialise a keras model and defining Binary Cross Entropy to calculate loss as well as calculating initial loss , accuracy

In [3]:
from tensorflow.keras.losses import BinaryCrossentropy
from sklearn.metrics import accuracy_score

# Loss function object
loss_fn = BinaryCrossentropy()

# Predict on test set before training
initial_preds = model(X_test)
initial_loss = loss_fn(y_test, initial_preds).numpy()

# Convert logits (probabilities) to 0/1 predictions
initial_class_preds = tf.cast(initial_preds > 0.5, tf.int32).numpy().flatten()
y_test_np = y_test.numpy().astype(int)

# Accuracy before training
initial_accuracy = accuracy_score(y_test_np, initial_class_preds)

# Predict on a risky customer (edge case)
new_customer = pd.DataFrame([{
    'age': 29,
    'income': 25000,
    'credit_score': 490,
    'loan_amount': 450000,
    'loan_term': 60,
    'past_defaults': 3
}])
new_customer_scaled = scaler.transform(new_customer)
X_new = tf.constant(new_customer_scaled, dtype=tf.float32)

# Probability of default before training
prob_before = model(X_new).numpy()[0][0]

# Final printouts
print(f"BEFORE Training → Probability of default (edge case): {prob_before:.4f}")
print(f"BEFORE Training → Loss on test set: {initial_loss:.4f}")
print(f"BEFORE Training → Accuracy on test set: {initial_accuracy * 100:.2f}%")


BEFORE Training → Probability of default (edge case): 0.4607
BEFORE Training → Loss on test set: 0.6920
BEFORE Training → Accuracy on test set: 55.00%


/usr/local/lib/python3.11/dist-packages/sklearn/utils/validation.py:2732: UserWarning: X has feature names, but MinMaxScaler was fitted without feature names
  warnings.warn(


# Running multiple epochs

In [5]:
model.compile(
    optimizer=tf.keras.optimizers.Adam(learning_rate=0.01),
    loss='binary_crossentropy',
    metrics=['accuracy']
)

history = model.fit(
    X_train,
    y_train,
    epochs=100,
    batch_size=32,
    validation_split=0.2,
    verbose=0  # Change to 1 if you want epoch logs
)
print("Training complete.")


Training complete.


In [ ]:
# Predict again on same input after training
prob_after = model(X_new).numpy()[0][0]
print(f"AFTER Training → Probability of default: {prob_after:.4f}")


✅ AFTER Training → Probability of default: 0.9958


In [ ]:
loss, accuracy = model.evaluate(X_test, y_test, verbose=0)
print(f"Test Loss: {loss:.4f}")
print(f"Test Accuracy: {accuracy * 100:.2f}%")


🧪 Test Loss: 0.2170
🎯 Test Accuracy: 91.50%
